## 3. *Encoder-Decoder* sobre Texto

Se elige el inglés al español ya que es mas fácil comprobar resultados de forma natural.

In [1]:
from keras import backend as K
import os
import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.python.framework import ops

# Set the environment variables
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Double check that you have the correct devices visible to TF
print("{0}\nThe available CPU/GPU devices on your system\n{0}".format('=' * 100))
print(device_lib.list_local_devices())

# Different options to start with GPU or CPU. I am using the CPU. Can be changed from the below options
# with tf.device('/cpu:0'):
with tf.device('/gpu:0'):
# with tf.Graph().as_default():
    config = tf.ConfigProto(device_count={'GPU': 1}, log_device_placement=False,
                            allow_soft_placement=True)
    # allocate only as much GPU memory based on runtime allocations
    config.gpu_options.allow_growth = True

Using TensorFlow backend.


The available CPU/GPU devices on your system
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17451588044000884234
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3150367948
locality {
  bus_id: 1
  links {
  }
}
incarnation: 3576444456178468212
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:08:00.0, compute capability: 6.1"
]


a) Visualice los datos ¿Qué es la entrada y qué es la salida? Comente sobre los múltiples significados/sinónimos que puede tener una palabra al ser traducida y cómo propondría arreglar eso. *se espera que pueda implementarlo*

Multiples significados/sinónimos para una palabra va a confundir a la red. Una forma sencilla es simplemente eliminar las filas duplicadas y solo dejar la primera ocurrencia de la palabra, sin embargo algunas tienen símbolos que las diferencian que mas adelante serán eliminados por lo que el paso de eliminar frases duplicadas se hará mas adelante.

In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv("data/spa.txt", sep="\t", names=["Source","Target"])
print(df.size)
# df[9500:9550]
# df[6200:6250]
df[:50]

246752


,Source,Target
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.
5,Run!,¡Corre!
6,Run!,¡Corran!
7,Run!,¡Corra!
8,Run!,¡Corred!
9,Run.,Corred.


b) Realice un pre-procesamiento a los textos como se acostumbra para eliminar símbolos inecesarios u otras cosas que estime conveniente, comente sobre la importancia de éste paso. Además de ésto deberá agregar un símbolo al final de la sentencia target para indicar un "alto" cuando la red neuronal necesite aprender a generar una sentencia.

En este paso se eliminan símbolos, números y puntuaciones, además con el texto simplificado es mas fácil detectar frases duplicadas como "Go now", de forma que cada frase de source ahora tiene una frase de target único, con esto se espera que la red no confunda palabras iguales en distintos contextos. Con las frases limpias ahora es posible crear un diccionario evitando palabras duplicadas.

In [ ]:
import nltk
nltk.download('punkt')

In [3]:
import string
import nltk
from nltk.stem import WordNetLemmatizer
table = str.maketrans('', '', string.punctuation) 
lemmatizer = WordNetLemmatizer() 
lemmaDict_es = {}
with open('data/lemmatization-es.txt', 'rb') as f:
    data = f.read().decode('utf8').replace(u'\r',u'').split(u'\n')
    data = [a.split(u'\t') for a in data]
    
for a in data:
    if len(a) > 1:
        lemmaDict_es[a[1]] = a[0]

def lemmatize(word):
    return lemmaDict_es.get(word, word)

def clean_text(text, where=None):
    """ OJO: Sin eliminar el significado de las palabras."""
    text = text.lower()
    tokenize_text = nltk.word_tokenize(text)
    tokenize_text = [''.join(c for c in word if c.isalnum()) for word in tokenize_text]#eliminar puntuacion
#     tokenize_text = [word.translate(table) for word in tokenize_text]#eliminar puntuacion# elimina palabras que empiezan con ! ej. !Ataque!
    tokenize_text = [word for word in tokenize_text if word.isalpha()] #remove numbers
#     tokenize_text = [lemmatizer.lemmatize(word) for word in tokenize_text]
    if where =="target":
#         tokenize_text = [lemmatize(word) for word in tokenize_text]
        tokenize_text = tokenize_text + ["#end"] 
    return tokenize_text
texts_input = list(df['Source'].apply(clean_text))
texts_output = list(df['Target'].apply(clean_text, where='target'))

In [4]:
dict = {}
texts_input_no_duplicates = list()
texts_output_no_duplicates = list()
for index, sentence_list in enumerate(texts_input):
    if str(sentence_list) not in dict:
        texts_input_no_duplicates.append(sentence_list)
        texts_output_no_duplicates.append(texts_output[index])
        dict[str(sentence_list)] = True

In [5]:
print(texts_input[:50])

[['go'], ['go'], ['go'], ['go'], ['hi'], ['run'], ['run'], ['run'], ['run'], ['run'], ['who'], ['wow'], ['fire'], ['fire'], ['fire'], ['help'], ['help'], ['help'], ['jump'], ['jump'], ['stop'], ['stop'], ['stop'], ['wait'], ['wait'], ['go', 'on'], ['go', 'on'], ['hello'], ['i', 'ran'], ['i', 'ran'], ['i', 'try'], ['i', 'won'], ['oh', 'no'], ['relax'], ['shoot'], ['shoot'], ['shoot'], ['shoot'], ['shoot'], ['shoot'], ['smile'], ['attack'], ['attack'], ['attack'], ['attack'], ['attack'], ['get', 'up'], ['go', 'now'], ['go', 'now'], ['go', 'now']]


In [6]:
print(texts_input_no_duplicates[:50])

[['go'], ['hi'], ['run'], ['who'], ['wow'], ['fire'], ['help'], ['jump'], ['stop'], ['wait'], ['go', 'on'], ['hello'], ['i', 'ran'], ['i', 'try'], ['i', 'won'], ['oh', 'no'], ['relax'], ['shoot'], ['smile'], ['attack'], ['get', 'up'], ['go', 'now'], ['got', 'it'], ['he', 'ran'], ['hop', 'in'], ['hug', 'me'], ['i', 'fell'], ['i', 'know'], ['i', 'left'], ['i', 'lied'], ['i', 'lost'], ['i', 'quit'], ['i', 'sang'], ['i', 'work'], ['i', 'm'], ['i', 'm', 'up'], ['listen'], ['no', 'way'], ['really'], ['thanks'], ['try', 'it'], ['we', 'try'], ['we', 'won'], ['why', 'me'], ['ask', 'tom'], ['awesome'], ['be', 'calm'], ['be', 'cool'], ['be', 'fair'], ['be', 'good']]


In [7]:
print(texts_output[:50])

[['ve', '#end'], ['vete', '#end'], ['vaya', '#end'], ['váyase', '#end'], ['hola', '#end'], ['corre', '#end'], ['corran', '#end'], ['corra', '#end'], ['corred', '#end'], ['corred', '#end'], ['quién', '#end'], ['órale', '#end'], ['fuego', '#end'], ['incendio', '#end'], ['disparad', '#end'], ['ayuda', '#end'], ['socorro', 'auxilio', '#end'], ['auxilio', '#end'], ['salta', '#end'], ['salte', '#end'], ['parad', '#end'], ['para', '#end'], ['pare', '#end'], ['espera', '#end'], ['esperen', '#end'], ['continúa', '#end'], ['continúe', '#end'], ['hola', '#end'], ['corrí', '#end'], ['corría', '#end'], ['lo', 'intento', '#end'], ['he', 'ganado', '#end'], ['oh', 'no', '#end'], ['tomátelo', 'con', 'soda', '#end'], ['fuego', '#end'], ['disparad', '#end'], ['disparen', '#end'], ['dispara', '#end'], ['dispará', '#end'], ['dispare', '#end'], ['sonríe', '#end'], ['al', 'ataque', '#end'], ['atacad', '#end'], ['ataque', '#end'], ['ataquen', '#end'], ['ataca', '#end'], ['levanta', '#end'], ['ve', 'ahora', 'm

In [8]:
print(texts_output_no_duplicates[:50])

[['ve', '#end'], ['hola', '#end'], ['corre', '#end'], ['quién', '#end'], ['órale', '#end'], ['fuego', '#end'], ['ayuda', '#end'], ['salta', '#end'], ['parad', '#end'], ['espera', '#end'], ['continúa', '#end'], ['hola', '#end'], ['corrí', '#end'], ['lo', 'intento', '#end'], ['he', 'ganado', '#end'], ['oh', 'no', '#end'], ['tomátelo', 'con', 'soda', '#end'], ['fuego', '#end'], ['sonríe', '#end'], ['al', 'ataque', '#end'], ['levanta', '#end'], ['ve', 'ahora', 'mismo', '#end'], ['lo', 'tengo', '#end'], ['él', 'corrió', '#end'], ['métete', 'adentro', '#end'], ['abrázame', '#end'], ['me', 'caí', '#end'], ['yo', 'lo', 'sé', '#end'], ['salí', '#end'], ['mentí', '#end'], ['perdí', '#end'], ['dimito', '#end'], ['canté', '#end'], ['estoy', 'trabajando', '#end'], ['tengo', 'diecinueve', '#end'], ['estoy', 'levantado', '#end'], ['escucha', '#end'], ['no', 'puede', 'ser', '#end'], ['en', 'serio', '#end'], ['gracias', '#end'], ['pruébalo', '#end'], ['lo', 'procuramos', '#end'], ['ganamos', '#end'], [

Cree un conjunto de validación y de pruebas fijos de Nexp=10000 datos ¿Cuántos datos quedan para entrenar? 

In [9]:
from sklearn.model_selection import train_test_split
N_exp = 10000
X_train_l, X_test_l, Y_train_l, Y_test_l = train_test_split(texts_input_no_duplicates, texts_output_no_duplicates,
                                                            test_size=N_exp, random_state=22)
X_train_l, X_val_l, Y_train_l, Y_val_l = train_test_split(X_train_l, Y_train_l, 
                                                          test_size=N_exp, random_state=22)
print("Datos de entrenamiento X: {}".format(len(X_train_l)))
print("Datos de entrenamiento Y: {}".format(len(Y_train_l)))
print("Datos de validacion X: {}".format((len(X_val_l))))
print("Datos de validacion Y: {}".format((len(Y_val_l))))
print("Datos de test X: {}".format((len(X_test_l))))
print("Datos de test Y: {}".format((len(Y_test_l))))
print("Frases de source: \n{}".format(X_train_l[:15]))
print("Frases de target: \n{}".format(Y_train_l[:15]))

Datos de entrenamiento X: 85107
Datos de entrenamiento Y: 85107
Datos de validacion X: 10000
Datos de validacion Y: 10000
Datos de test X: 10000
Datos de test Y: 10000
Frases de source: 
[['tom', 'introduced', 'me', 'to', 'his', 'roommate'], ['it', 's', 'really', 'not', 'as', 'hard', 'to', 'do', 'as', 'you', 'think', 'it', 'is'], ['i', 'could', 'nt', 'stand', 'the', 'itching'], ['tom', 'brought', 'mary', 'to', 'the', 'station'], ['was', 'it', 'all', 'an', 'illusion'], ['tom', 'is', 'good', 'at', 'skiing'], ['i', 'like', 'the', 'beach'], ['he', 's', 'a', 'strange', 'guy'], ['are', 'nt', 'you', 'anxious'], ['how', 'did', 'that', 'happen', 'again'], ['shall', 'i', 'begin'], ['i', 'think', 'we', 'should', 'lower', 'the', 'price'], ['he', 'was', 'sentenced', 'to', 'prison'], ['he', 'touched', 'me', 'on', 'the', 'cheek'], ['the', 'wound', 'has', 'not', 'healed', 'yet']]
Frases de target: 
[['tom', 'me', 'presentó', 'a', 'su', 'compañero', 'de', 'habitación', '#end'], ['si', 'no', 'es', 'tan'

In [10]:
import matplotlib.pyplot as plt 
en_l = []
es_l = []
for i in X_train_l:
    en_l.append(len(i))
for i in Y_train_l:
    es_l.append(len(i))
df_lengths = pd.DataFrame({'en':en_l, 'es':es_l})
df_lengths.hist(bins=20)
plt.show()

<Figure size 640x480 with 2 Axes>

c) Genere un vocabulario, desde el conjunto de entrenamiento, sobre las palabras a recibir y generar en la traducción, esto es codificarlas a un valor entero que servirá para que la red las vea en una representación útil a procesar, comience desde el 1 debido a que el cero será utilizado más adelante. Para reducir el vocabulario considere las palabras que aparecen un mínimo de min_count veces en todo los datos, se aconseja un valor de 3. Comente sobre la importancia de ésto al reducir el vocabulario ¿De qué tamaño es el vocabulario de entrada y salida? ¿La diferencia de ésto podría ser un factor importante?

In [11]:
def create_vocab(texts, min_count=1):
    count_vocab = {}
    for sentence in texts:
        for word in sentence:
            if word not in count_vocab:
                count_vocab[word] = 1
            else:
                count_vocab[word] += 1
    return [word for word,count in count_vocab.items() if count >= min_count]
vocab_source = create_vocab(X_train_l, min_count=3)
# vocab_source = create_vocab(texts_input, min_count=3)
word2idx_s = {w: i+1 for i, w in enumerate(vocab_source)} #index (i+1) start from 1,2,3,...
idx2word_s = {i+1: w for i, w in enumerate(vocab_source)}
n_words_s = len(vocab_source)
vocab_target = create_vocab(Y_train_l, min_count=3)
# vocab_target = create_vocab(texts_output, min_count=3)
word2idx_t = {w: i+1 for i, w in enumerate(vocab_target)}  #Converting text to numbers
idx2word_t = {i+1: w for i, w in enumerate(vocab_target)} #Converting number to text
n_words_t = len(vocab_target)
print("Diccionario de source: {}".format(n_words_s))
print("Diccionario de target: {}".format(n_words_t))
print("Primeras 15 entradas de source: \n{}".format(list(idx2word_s.items())[:15]))
print("Primeras 15 entradas de target: \n{}".format(list(idx2word_t.items())[:15]))

Diccionario de source: 6102
Diccionario de target: 9235
Primeras 15 entradas de source: 
[(1, 'tom'), (2, 'introduced'), (3, 'me'), (4, 'to'), (5, 'his'), (6, 'roommate'), (7, 'it'), (8, 's'), (9, 'really'), (10, 'not'), (11, 'as'), (12, 'hard'), (13, 'do'), (14, 'you'), (15, 'think')]
Primeras 15 entradas de target: 
[(1, 'tom'), (2, 'me'), (3, 'presentó'), (4, 'a'), (5, 'su'), (6, 'compañero'), (7, 'de'), (8, 'habitación'), (9, '#end'), (10, 'si'), (11, 'no'), (12, 'es'), (13, 'tan'), (14, 'difícil'), (15, 'como')]


La diferencia del tamaño del vocabulario implica que existen palabras en target que no tienen una traduccion directa en source, o que algunas palabras en source tienen múltiples significados en target, lo que ya se observo en la parte a)

Ahora codifique las palabras a los números indexados con el vocabulario. Recuerde que si una palabra en los otros conjuntos, o en el mismo de entrenamiento, no aparece en el vocabulario no se podrá generar una codificación, por lo que será ignorada ¿Cómo se podría evitar ésto?

In [12]:
""" Source/input data """
dataX_train = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_train_l]
dataX_valid = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_val_l]
dataX_test = [[word2idx_s[word] for word in sent if word in word2idx_s] for sent in X_test_l]
""" Target/output data """
dataY_train = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_train_l]
dataY_valid = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_val_l]
dataY_test = [[word2idx_t[word] for word in sent if word in word2idx_t] for sent in Y_test_l]
print(X_train_l[:15])
print(dataX_train[:15])
print(Y_train_l[:15])
print(dataY_train[:15])

[['tom', 'introduced', 'me', 'to', 'his', 'roommate'], ['it', 's', 'really', 'not', 'as', 'hard', 'to', 'do', 'as', 'you', 'think', 'it', 'is'], ['i', 'could', 'nt', 'stand', 'the', 'itching'], ['tom', 'brought', 'mary', 'to', 'the', 'station'], ['was', 'it', 'all', 'an', 'illusion'], ['tom', 'is', 'good', 'at', 'skiing'], ['i', 'like', 'the', 'beach'], ['he', 's', 'a', 'strange', 'guy'], ['are', 'nt', 'you', 'anxious'], ['how', 'did', 'that', 'happen', 'again'], ['shall', 'i', 'begin'], ['i', 'think', 'we', 'should', 'lower', 'the', 'price'], ['he', 'was', 'sentenced', 'to', 'prison'], ['he', 'touched', 'me', 'on', 'the', 'cheek'], ['the', 'wound', 'has', 'not', 'healed', 'yet']]
[[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12, 4, 13, 11, 14, 15, 7, 16], [17, 18, 19, 20, 21], [1, 22, 23, 4, 21, 24], [25, 7, 26, 27, 28], [1, 16, 29, 30, 31], [17, 32, 21, 33], [34, 8, 35, 36, 37], [38, 19, 14, 39], [40, 41, 42, 43, 44], [45, 17, 46], [17, 15, 47, 48, 49, 21, 50], [34, 25, 51, 4, 52], [34, 53,

d) Debido al largo variable de los textos de entrada y salida será necesario estandarizar ésto para poder trabajar de manera más cómoda en Keras, cada texto (entrada y salida) pueden tener distinto largo máximo. Comente sobre la decisión del tipo de padding, pre o post ¿Qué sucede al variar el largo máximo de instantes de tiempo para procesar en cada parte del modelo (entrada y salida)?

Dwarampudi & Reddy (2019). "Effects of padding on LSTMs and CNNs" [[1]](#refs)

Pre y post padding tienen gran importancia en redes LSTM ya que tienen memoria, la cual se actualiza a medida que se introducen las palabras de las frases y en un estudio fue comprobado que tienen mejores resultado utilizando pre padding. En cambio en redes CNN pre y post padding no tienen relevancia ya que estas redes no tienen memoria, sino que se ajustan a los datos entregados tratando de encontrar patrones. Por lo que finalmente es mejor utilizar pre padding ya que en el peor de los casos no tiene beneficios.

In [13]:
from keras.preprocessing import sequence
""" INPUT DATA (Origin language) """
#TRAIN
max_inp_length = max(map(len,dataX_train))
max_inp_length_val = max(map(len,dataX_valid))
max_inp_length_test = max(map(len,dataX_test))
max_inp_len = max(max_inp_length,max_inp_length_val,max_inp_length_test)
print("Largo max inp: ",max_inp_len)
word2idx_s["*"] = 0 #padding symbol
idx2word_s[0] = "*"
n_words_s += 1  
X_train = sequence.pad_sequences(dataX_train, maxlen=max_inp_len, padding='pre', value=word2idx_s["*"])
#VALID
# print("Largo max inp val: ",max_inp_length_val)
X_val = sequence.pad_sequences(dataX_valid, maxlen=max_inp_len, padding='pre', value=word2idx_s["*"])
#TEST
# print("Largo max inp test: ",max_inp_length_test)
X_test = sequence.pad_sequences(dataX_test, maxlen=max_inp_len, padding='pre', value=word2idx_s["*"])

""" OUTPUT DATA (Destination language) """
#TRAIN
max_out_length = max(map(len,dataY_train)) 
max_out_length_val = max(map(len,dataY_valid))
max_out_length_test = max(map(len,dataY_test))
max_out_len = max(max_out_length,max_out_length_val,max_out_length_test)
print("Largo max out: ",max_out_len)
word2idx_t["*"] = 0 #padding symbol
idx2word_t[0] = "*"
n_words_t += 1  
Y_train = sequence.pad_sequences(dataY_train, maxlen=max_out_len, padding='post', value=word2idx_t["*"])
#VALID
# print("Largo max out val: ",max_out_length_val)
Y_val = sequence.pad_sequences(dataY_valid, maxlen=max_out_len, padding='post', value=word2idx_t["*"])
#TEST
# print("Largo max out test: ",max_out_length_test)
Y_test = sequence.pad_sequences(dataY_test, maxlen=max_out_len, padding='post', value=word2idx_t["*"])

Largo max inp:  45
Largo max out:  51


e) Para evitar que la red obtenga una ganancia por imitar/predecir el símbolo de padding que está bastante presente en los datos coloque un peso sobre éste clase, con valor 0, así se evita que tenga impacto en la función objetivo. Ya que keras no soporta directamente ésto en series de tiempo coloque el peso a cada instante de tiempo de cada dato de entrenamiento dependiendo de su clase. Comente sobre alguna otra forma en que se podría manejar el evitar que la red prediga en mayoría el símbolo de padding.

In [14]:
c_weights = np.ones(n_words_t)
c_weights[0] = 0 #padding class masked
sample_weight = np.zeros(Y_train.shape)
for i in range(sample_weight.shape[0]):
    sample_weight[i] = c_weights[Y_train[i,:]]

Se puede lograr el mismo efecto utilizando el parametro *mask_zero = True* en la capa de embedding de keras, la documentación de keras dice: <br>
>If mask_zero is set to True, the input value 0 will be a special "padding" that should be masked out.
Index 0 cannot be used in the vocabulary. (_Embedding Layer_ doc)

f) Para lograr la tarea defina una red recurrente del tipo encoder-decoder como la que se presenta en la siguiente imágen.
En primer lugar defina el Encoder que procesara el texto de entrada y retornará un solo vector final, haciendo uso de las capas ya conocidas de Embedding para generar un vector denso de palabra y GRU, pero en su versión acelerada para GPU.

In [15]:
#ENCODER
from keras.models import Sequential
from keras.layers import Embedding,CuDNNGRU
EMBEDDING_DIM = 100
model = Sequential()
model.add(Embedding(input_dim=n_words_s, output_dim=EMBEDDING_DIM, input_length=max_inp_len))
model.add(CuDNNGRU(64, return_sequences=True))
model.add(CuDNNGRU(128, return_sequences=False))

Instructions for updating:
Colocations handled automatically by placer.


In [16]:
from keras.layers import RepeatVector
model.add(RepeatVector(max_out_length)) #conection

In [17]:
#DECODER
from keras.layers import CuDNNGRU, TimeDistributed,Dense
model.add(CuDNNGRU(128, return_sequences=True))
model.add(CuDNNGRU(64, return_sequences=True))
model.add(TimeDistributed(Dense(n_words_t, activation='softmax')))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 45, 100)           610300    
_________________________________________________________________
cu_dnngru_1 (CuDNNGRU)       (None, 45, 64)            31872     
_________________________________________________________________
cu_dnngru_2 (CuDNNGRU)       (None, 128)               74496     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 51, 128)           0         
_________________________________________________________________
cu_dnngru_3 (CuDNNGRU)       (None, 51, 128)           99072     
_________________________________________________________________
cu_dnngru_4 (CuDNNGRU)       (None, 51, 64)            37248     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 51, 9236)          600340    
Total para

In [18]:
Y_train_exp = np.reshape(Y_train, (Y_train.shape[0], Y_train.shape[1],1))
Y_val_exp = np.reshape(Y_val, (Y_val.shape[0], Y_val.shape[1], 1))
print(Y_train_exp.shape)
print(Y_val_exp.shape)

(85107, 51, 1)
(10000, 51, 1)


In [19]:
%%time
BATCH_SIZE = 256
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', sample_weight_mode='temporal')
model.fit(X_train, Y_train_exp, epochs=3, batch_size=BATCH_SIZE,validation_data=(X_val, Y_val_exp),
         sample_weight = sample_weight) 

Instructions for updating:
Use tf.cast instead.
Train on 85107 samples, validate on 10000 samples
Epoch 1/3
85107/85107 [==============================] - 121s 1ms/step - loss: 6.2936 - val_loss: 12.5617
Epoch 2/3
85107/85107 [==============================] - 117s 1ms/step - loss: 6.0092 - val_loss: 13.5133
Epoch 3/3
85107/85107 [==============================] - 110s 1ms/step - loss: 5.9344 - val_loss: 14.0965
Wall time: 5min 47s


g) Debido a lo costoso de tener una red completamente recurrente para entrenar y poder experimentar, cambie el modelo que procesa el Encoder por una red convolucional, reduciendo el número de capas pero aumentando las neuronas. Utilice tamaños de kernel igual a 5 y funciones de activaciones relu. Se agregan capas de BatchNormalization debido a que en el Decoder contamos con redes recurrentes que tienen capa activación distinta a la usada por las convoluciones. La capa de GlobalMaxPooling1d es lo que permite reducir toda la información extraída a un único vector, como se realizó anteriormente con return_sequences=False, comente sobre la ganancia o desventaja de ésto vs la red neuronal.

In [20]:
from keras.layers import Conv1D,MaxPool1D,GlobalMaxPooling1D,GlobalAveragePooling1D,BatchNormalization
model = Sequential()
model.add(Embedding(input_dim=n_words_s, output_dim=EMBEDDING_DIM, input_length=max_inp_len))
model.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
model.add(BatchNormalization()) #for stability
model.add(Conv1D(256, 5, padding='same', activation='relu', strides=1))
model.add(BatchNormalization())
model.add(GlobalMaxPooling1D()) #aka to return_sequences=False
model.add(RepeatVector(max_out_length)) #conection
model.add(CuDNNGRU(256, return_sequences=True))
model.add(TimeDistributed(Dense(n_words_t, activation='softmax')))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 45, 100)           610300    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 45, 256)           128256    
_________________________________________________________________
batch_normalization_1 (Batch (None, 45, 256)           1024      
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 45, 256)           327936    
_________________________________________________________________
batch_normalization_2 (Batch (None, 45, 256)           1024      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 256)               0         
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 51, 256)           0         
__________

In [21]:
%%time
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', sample_weight_mode='temporal')
model.fit(X_train, Y_train_exp, epochs=20, batch_size=256,validation_data=(X_val, Y_val_exp),
         sample_weight = sample_weight) 

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 85107 samples, validate on 10000 samples
Epoch 1/20
85107/85107 [==============================] - 162s 2ms/step - loss: 5.6565 - val_loss: 14.3453
Epoch 2/20
85107/85107 [==============================] - 149s 2ms/step - loss: 4.6114 - val_loss: 14.5112
Epoch 3/20
85107/85107 [==============================] - 144s 2ms/step - loss: 4.0088 - val_loss: 14.4619
Epoch 4/20
85107/85107 [==============================] - 144s 2ms/step - loss: 3.5915 - val_loss: 14.4253
Epoch 5/20
85107/85107 [==============================] - 145s 2ms/step - loss: 3.2682 - val_loss: 14.3974
Epoch 6/20
85107/85107 [==============================] - 144s 2ms/step - loss: 3.0092 - val_loss: 14.3700
Epoch 7/20
85107/85107 [==============================] - 147s 2ms/step - loss: 2.8132 - val_loss: 14.3655
Epoch 8/20
85107/85107 [==============================] - 144s 2ms/step - loss: 2.6352 - val_loss: 14.3485
Epoch 9/20
85107

h) Visualice lo aprendido por el modelo sobre algunos datos del conjunto de entrenamiento y validación, comente lo observado.

In [ ]:
def predict_words(y_indexs, data="target"):
    """ Predict until '-#end-' is seen """
    return_val = []
    for indx_word in y_indexs:
        if indx_word != 0: #start to predict
            return_val.append(np.squeeze(indx_word))
            if data == "target": #if target is predicting
                if indx_word == word2idx_t["#end"]:
                    return return_val                
    return return_val
n_s = 100
idx = np.random.choice(np.arange(Y_set.shape[0]), size=n_s, replace=False)
Y_set_pred = model.predict_classes(X_set[idx] )
for i, n_sampled in enumerate(idx):
    text_input = [idx2word_s[p] for p in predict_words(X_set[n_sampled], data="source")]
    print("Texto source: ", ' '.join(text_input))
    text_real = [idx2word_t[p] for p in predict_words(Y_set[n_sampled,:,0], data="target")]
    print("Texto target real: ", ' '.join( text_real))
    text_sampled = [idx2word_t[p] for p in predict_words(Y_set_pred[i], data="target")]
    print("Texto target predicho: ", ' '.join(text_sampled))

<a id="refs"></a>
## Referencias
[1] Dwarampudi, M., & Reddy, N. V. (2019). Effects of padding on LSTMs and CNNs. arXiv preprint [arXiv:1903.07288](https://arxiv.org/abs/1903.07288)